# 🔬 Spectroscopy & Biomedical Measurements: Hands-on Practice

## Table of Contents
1. [Energy-Wavelength-Frequency Calculations](#practice-1-energy-wavelength-frequency-calculations)
2. [Beer-Lambert Law and UV-Vis Spectroscopy](#practice-2-beer-lambert-law-and-uv-vis-spectroscopy)
3. [Protein Concentration Measurement](#practice-3-protein-concentration-measurement)
4. [DNA/RNA Quantification and Purity](#practice-4-dnarna-quantification-and-purity)
5. [Fluorescence Spectra Analysis](#practice-5-fluorescence-spectra-analysis)
6. [FRET Efficiency Calculation](#practice-6-fret-efficiency-calculation)
7. [Spectral Data Processing](#practice-7-spectral-data-processing)
8. [Flow Cytometry Data Simulation](#practice-8-flow-cytometry-data-simulation)

## Installing and Importing Essential Libraries

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')
sns.set_palette('husl')

print("✅ All libraries loaded successfully!")
print("📊 Ready for spectroscopy data analysis!")

---
## Practice 1: Energy-Wavelength-Frequency Calculations

### 🎯 Learning Objectives
- Apply the Planck-Einstein relation: **E = hν = hc/λ**
- Convert between wavelength, frequency, and photon energy
- Calculate biological energy scales

### 📖 Key Concepts
- **Planck constant:** h = 6.626 × 10⁻³⁴ J·s
- **Speed of light:** c = 3 × 10⁸ m/s
- **Energy in eV:** E(eV) = 1240 / λ(nm)

In [ ]:
# 1.1 Define physical constants
h = 6.626e-34  # Planck constant (J·s)
c = 3.0e8      # Speed of light (m/s)
eV_to_J = 1.602e-19  # Conversion factor

def wavelength_to_energy(wavelength_nm):
    """Convert wavelength (nm) to energy (eV)"""
    wavelength_m = wavelength_nm * 1e-9
    energy_J = (h * c) / wavelength_m
    energy_eV = energy_J / eV_to_J
    return energy_eV

def wavelength_to_frequency(wavelength_nm):
    """Convert wavelength (nm) to frequency (Hz)"""
    wavelength_m = wavelength_nm * 1e-9
    frequency = c / wavelength_m
    return frequency

# Calculate for biological relevant wavelengths
wavelengths = {
    'UV-C (germicidal)': 254,
    'UV-A (blacklight)': 365,
    'Blue light': 450,
    'Green (GFP emission)': 509,
    'Red (mCherry)': 610,
    'Near-IR': 850
}

print("🌈 Biological Wavelengths: Energy and Frequency")
print("=" * 70)
print(f"{'Region':<25} {'λ (nm)':<12} {'Energy (eV)':<15} {'Frequency (THz)'}")
print("=" * 70)

for region, wl in wavelengths.items():
    energy = wavelength_to_energy(wl)
    freq = wavelength_to_frequency(wl) / 1e12  # Convert to THz
    print(f"{region:<25} {wl:<12} {energy:<15.3f} {freq:.2f}")

print("\n💡 Key Insight: Higher energy (shorter wavelength) can damage biomolecules!")

In [ ]:
# 1.2 Visualize EM spectrum energy scales
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Energy vs wavelength
wavelengths_range = np.linspace(200, 1000, 500)
energies = [wavelength_to_energy(wl) for wl in wavelengths_range]

ax1.plot(wavelengths_range, energies, linewidth=2.5, color='#1E64C8')
ax1.set_xlabel('Wavelength (nm)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Photon Energy (eV)', fontsize=12, fontweight='bold')
ax1.set_title('Energy-Wavelength Relationship: E = hc/λ', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axvline(280, color='purple', linestyle='--', alpha=0.7, label='Protein (280 nm)')
ax1.axvline(509, color='green', linestyle='--', alpha=0.7, label='GFP (509 nm)')
ax1.legend()

# Biological energy scales
bio_energies = {
    'Thermal energy\n(kBT at 25°C)': 0.026,
    'IR vibrations\n(C-H, N-H bonds)': 0.1,
    'Visible light\n(photosynthesis)': 2.0,
    'UV damage\n(DNA breaks)': 4.5
}

labels = list(bio_energies.keys())
values = list(bio_energies.values())
colors = ['#FFD700', '#FFA500', '#27ae60', '#E74C3C']

bars = ax2.barh(labels, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
ax2.set_title('Biological Energy Scales', fontsize=14, fontweight='bold')
ax2.grid(True, axis='x', alpha=0.3)

# Add value labels
for bar, val in zip(bars, values):
    ax2.text(val + 0.1, bar.get_y() + bar.get_height()/2, 
             f'{val:.3f} eV', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("📈 Graph shows inverse relationship: shorter wavelength → higher energy")

---
## Practice 2: Beer-Lambert Law and UV-Vis Spectroscopy

### 🎯 Learning Objectives
- Apply Beer-Lambert Law: **A = εbc**
- Calculate absorbance and transmittance
- Determine unknown concentrations

### 📖 Key Formula
**A = εbc = -log₁₀(I/I₀)**
- A: Absorbance (unitless)
- ε: Molar absorptivity (M⁻¹cm⁻¹)
- b: Path length (cm)
- c: Concentration (M)

In [ ]:
# 2.1 Beer-Lambert Law calculations
def calculate_absorbance(epsilon, path_length, concentration):
    """Calculate absorbance using Beer-Lambert Law"""
    return epsilon * path_length * concentration

def calculate_concentration(absorbance, epsilon, path_length):
    """Calculate concentration from absorbance"""
    return absorbance / (epsilon * path_length)

def absorbance_to_transmittance(absorbance):
    """Convert absorbance to % transmittance"""
    return 10**(-absorbance) * 100

# Example: NADH measurement
epsilon_NADH = 6220  # M⁻¹cm⁻¹ at 340 nm
path_length = 1.0    # cm (standard cuvette)
concentration = 50e-6  # 50 μM = 50 × 10⁻⁶ M

absorbance = calculate_absorbance(epsilon_NADH, path_length, concentration)
transmittance = absorbance_to_transmittance(absorbance)

print("📊 Beer-Lambert Law Example: NADH Measurement")
print("=" * 60)
print(f"Molar absorptivity (ε): {epsilon_NADH} M⁻¹cm⁻¹")
print(f"Path length (b): {path_length} cm")
print(f"Concentration (c): {concentration*1e6:.1f} μM")
print(f"\n➡️  Absorbance (A): {absorbance:.4f}")
print(f"➡️  Transmittance (T): {transmittance:.2f}%")
print(f"\n✓ A = {absorbance:.4f} is in the optimal linear range (0.1-1.0)")

In [ ]:
# 2.2 Generate and visualize calibration curve
np.random.seed(42)

# Generate calibration data
concentrations_uM = np.array([0, 10, 25, 50, 75, 100])  # μM
concentrations_M = concentrations_uM * 1e-6

# Calculate theoretical absorbances
absorbances_theoretical = epsilon_NADH * path_length * concentrations_M

# Add experimental noise
noise = np.random.normal(0, 0.01, len(absorbances_theoretical))
absorbances_measured = absorbances_theoretical + noise
absorbances_measured = np.maximum(absorbances_measured, 0)  # No negative absorbance

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(concentrations_uM, absorbances_measured)

# Unknown sample
unknown_absorbance = 0.45
unknown_concentration = (unknown_absorbance - intercept) / slope

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Plot calibration curve
ax.scatter(concentrations_uM, absorbances_measured, s=100, color='#1E64C8', 
           edgecolor='black', linewidth=2, label='Measured data', zorder=3)

# Fit line
x_fit = np.linspace(0, 110, 100)
y_fit = slope * x_fit + intercept
ax.plot(x_fit, y_fit, 'r--', linewidth=2, label=f'Linear fit (R² = {r_value**2:.4f})', zorder=2)

# Unknown sample
ax.axhline(unknown_absorbance, color='green', linestyle=':', linewidth=2, alpha=0.7, label='Unknown sample')
ax.axvline(unknown_concentration, color='green', linestyle=':', linewidth=2, alpha=0.7)
ax.plot(unknown_concentration, unknown_absorbance, 'g*', markersize=20, 
        markeredgecolor='black', markeredgewidth=1.5, zorder=4)

# Labels and formatting
ax.set_xlabel('Concentration (μM)', fontsize=13, fontweight='bold')
ax.set_ylabel('Absorbance at 340 nm', fontsize=13, fontweight='bold')
ax.set_title('NADH Calibration Curve (Beer-Lambert Law)', fontsize=15, fontweight='bold')
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3)

# Add equation
equation_text = f'A = {slope:.4f} × C + {intercept:.4f}'
ax.text(0.98, 0.05, equation_text, transform=ax.transAxes, 
        fontsize=12, verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n🎯 Unknown Sample Analysis:")
print(f"   Measured absorbance: {unknown_absorbance:.4f}")
print(f"   ➡️  Calculated concentration: {unknown_concentration:.2f} μM")
print(f"\n✅ Calibration quality: R² = {r_value**2:.4f} (excellent linear fit!)")

---
## Practice 3: Protein Concentration Measurement

### 🎯 Learning Objectives
- Calculate protein concentration using A280 method
- Compare Bradford and BCA assays
- Apply extinction coefficient calculations

### 📖 Key Methods
- **A280 method:** Direct measurement, needs pure protein
- **Bradford:** Coomassie dye binding (1-100 μg/mL)
- **BCA:** Cu²⁺ reduction, detergent compatible (20-2000 μg/mL)

In [ ]:
# 3.1 A280 method for protein quantification
def calculate_protein_concentration_A280(absorbance_280, extinction_coeff, path_length=1.0):
    """
    Calculate protein concentration using A280 method
    c (mg/mL) = A280 / ε (for 1 cm path length)
    """
    concentration = absorbance_280 / extinction_coeff
    return concentration

# Example proteins with different extinction coefficients
proteins = {
    'BSA (Bovine Serum Albumin)': {'ε': 0.667, 'MW': 66463},  # (mg/mL)⁻¹cm⁻¹
    'Lysozyme': {'ε': 2.64, 'MW': 14300},
    'GFP': {'ε': 0.21, 'MW': 27000},
    'IgG (antibody)': {'ε': 1.35, 'MW': 150000}
}

# Measured absorbances
measured_A280 = 0.85

print("🧬 Protein Concentration Calculation (A280 Method)")
print("=" * 70)
print(f"Measured absorbance at 280 nm: {measured_A280}")
print("\n" + "=" * 70)
print(f"{'Protein':<30} {'ε₂₈₀':<15} {'Conc (mg/mL)':<15} {'Conc (μM)'}")
print("=" * 70)

for protein_name, props in proteins.items():
    conc_mg_ml = calculate_protein_concentration_A280(measured_A280, props['ε'])
    conc_uM = (conc_mg_ml / props['MW']) * 1e6  # Convert to μM
    print(f"{protein_name:<30} {props['ε']:<15.3f} {conc_mg_ml:<15.3f} {conc_uM:.2f}")

print("\n💡 Note: Same A280 gives different concentrations due to different ε values!")

In [ ]:
# 3.2 Bradford Assay calibration
np.random.seed(123)

# Bradford assay standard curve (BSA standards)
bsa_standards_ug_ml = np.array([0, 5, 10, 25, 50, 75, 100])  # μg/mL
absorbance_595 = np.array([0.02, 0.15, 0.28, 0.62, 1.05, 1.38, 1.65])

# Add slight noise
absorbance_595_noisy = absorbance_595 + np.random.normal(0, 0.02, len(absorbance_595))

# Fit standard curve (polynomial fit for Bradford - it's non-linear!)
coeffs = np.polyfit(bsa_standards_ug_ml, absorbance_595_noisy, 2)
poly_func = np.poly1d(coeffs)

# Unknown samples
unknown_samples = {
    'Sample A': 0.45,
    'Sample B': 0.92,
    'Sample C': 1.25
}

# Solve for concentrations (quadratic equation)
def bradford_conc_from_abs(absorbance, coeffs):
    """Solve quadratic equation to find concentration"""
    a, b, c = coeffs[0], coeffs[1], coeffs[2] - absorbance
    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        return None
    conc = (-b + np.sqrt(discriminant)) / (2*a)
    return max(conc, 0)

# Visualization
fig, ax = plt.subplots(figsize=(11, 6))

# Plot standards
ax.scatter(bsa_standards_ug_ml, absorbance_595_noisy, s=120, color='#1E64C8',
           edgecolor='black', linewidth=2, label='BSA standards', zorder=3)

# Plot fit curve
x_smooth = np.linspace(0, 110, 200)
y_smooth = poly_func(x_smooth)
ax.plot(x_smooth, y_smooth, 'r-', linewidth=2.5, label='Quadratic fit', zorder=2)

# Plot unknown samples
colors_samples = ['green', 'orange', 'purple']
for (sample_name, abs_val), color in zip(unknown_samples.items(), colors_samples):
    conc = bradford_conc_from_abs(abs_val, coeffs)
    if conc is not None:
        ax.plot(conc, abs_val, 'o', markersize=12, color=color, 
                markeredgecolor='black', markeredgewidth=2, label=sample_name, zorder=4)
        ax.plot([conc, conc], [0, abs_val], ':', color=color, linewidth=1.5, alpha=0.7)

ax.set_xlabel('BSA Concentration (μg/mL)', fontsize=13, fontweight='bold')
ax.set_ylabel('Absorbance at 595 nm', fontsize=13, fontweight='bold')
ax.set_title('Bradford Assay Standard Curve', fontsize=15, fontweight='bold')
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(-5, 110)
ax.set_ylim(0, 1.8)

plt.tight_layout()
plt.show()

print("\n🧪 Unknown Sample Concentrations (Bradford Assay):")
print("=" * 50)
for sample_name, abs_val in unknown_samples.items():
    conc = bradford_conc_from_abs(abs_val, coeffs)
    print(f"{sample_name}: A₅₉₅ = {abs_val:.3f} → Concentration = {conc:.2f} μg/mL")

print("\n⚠️  Note: Bradford assay shows non-linear response at higher concentrations!")

---
## Practice 4: DNA/RNA Quantification and Purity

### 🎯 Learning Objectives
- Quantify DNA/RNA using A260
- Calculate purity ratios (A260/A280, A260/A230)
- Interpret contamination indicators

### 📖 Key Standards
- **Pure DNA:** A260/A280 ≈ 1.8
- **Pure RNA:** A260/A280 ≈ 2.0
- **Protein contamination:** A260/A280 < 1.8
- **Salt/organic contamination:** A260/A230 < 2.0

In [ ]:
# 4.1 DNA/RNA quantification
def calculate_nucleic_acid_concentration(A260, dilution_factor, na_type='DNA'):
    """
    Calculate nucleic acid concentration
    1 A260 unit = 50 μg/mL for dsDNA
    1 A260 unit = 40 μg/mL for RNA
    1 A260 unit = 33 μg/mL for ssDNA
    """
    conversion_factors = {
        'DNA': 50,
        'RNA': 40,
        'ssDNA': 33
    }
    
    factor = conversion_factors.get(na_type, 50)
    concentration = A260 * factor * dilution_factor
    return concentration

def assess_purity(A260, A280, A230):
    """Assess nucleic acid purity from absorbance ratios"""
    ratio_260_280 = A260 / A280
    ratio_260_230 = A260 / A230
    
    # Purity assessment for DNA
    if 1.7 <= ratio_260_280 <= 1.9:
        purity_280 = "✅ Pure (no protein contamination)"
    elif ratio_260_280 < 1.7:
        purity_280 = "⚠️  Protein contamination detected"
    else:
        purity_280 = "⚠️  RNA or phenol contamination"
    
    if ratio_260_230 >= 2.0:
        purity_230 = "✅ Pure (no salt/organic contamination)"
    else:
        purity_230 = "⚠️  Salt or organic solvent contamination"
    
    return ratio_260_280, ratio_260_230, purity_280, purity_230

# Sample measurements
samples_data = {
    'Genomic DNA (pure)': {'A260': 0.875, 'A280': 0.485, 'A230': 0.420, 'dilution': 50},
    'Plasmid DNA': {'A260': 1.250, 'A280': 0.690, 'A230': 0.615, 'dilution': 100},
    'Total RNA (pure)': {'A260': 1.050, 'A280': 0.520, 'A230': 0.510, 'dilution': 50},
    'Contaminated DNA': {'A260': 0.680, 'A280': 0.545, 'A230': 0.750, 'dilution': 50},
}

print("🧬 DNA/RNA Quantification and Purity Assessment")
print("=" * 90)

results = []
for sample_name, data in samples_data.items():
    # Determine NA type from name
    na_type = 'RNA' if 'RNA' in sample_name else 'DNA'
    
    # Calculate concentration
    conc = calculate_nucleic_acid_concentration(data['A260'], data['dilution'], na_type)
    
    # Assess purity
    r_260_280, r_260_230, pur_280, pur_230 = assess_purity(data['A260'], data['A280'], data['A230'])
    
    results.append({
        'Sample': sample_name,
        'Concentration': conc,
        '260/280': r_260_280,
        '260/230': r_260_230,
        'Purity_280': pur_280,
        'Purity_230': pur_230
    })
    
    print(f"\n📊 {sample_name}")
    print(f"   Concentration: {conc:.1f} μg/mL")
    print(f"   A260/A280 = {r_260_280:.3f} → {pur_280}")
    print(f"   A260/A230 = {r_260_230:.3f} → {pur_230}")

print("\n" + "=" * 90)

In [ ]:
# 4.2 Visualize purity ratios
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Extract data for plotting
sample_names = [r['Sample'] for r in results]
ratios_260_280 = [r['260/280'] for r in results]
ratios_260_230 = [r['260/230'] for r in results]
concentrations = [r['Concentration'] for r in results]

# Plot 1: A260/A280 ratios
colors_1 = ['green' if 1.7 <= r <= 1.9 else 'red' for r in ratios_260_280]
bars1 = ax1.barh(sample_names, ratios_260_280, color=colors_1, alpha=0.7, edgecolor='black', linewidth=2)
ax1.axvline(1.8, color='blue', linestyle='--', linewidth=2, label='Pure DNA (1.8)')
ax1.axvline(2.0, color='purple', linestyle='--', linewidth=2, label='Pure RNA (2.0)')
ax1.axvspan(1.7, 1.9, alpha=0.2, color='green', label='Acceptable range')
ax1.set_xlabel('A260/A280 Ratio', fontsize=12, fontweight='bold')
ax1.set_title('Purity Assessment: Protein Contamination', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, axis='x', alpha=0.3)

# Plot 2: A260/A230 ratios
colors_2 = ['green' if r >= 2.0 else 'orange' for r in ratios_260_230]
bars2 = ax2.barh(sample_names, ratios_260_230, color=colors_2, alpha=0.7, edgecolor='black', linewidth=2)
ax2.axvline(2.0, color='blue', linestyle='--', linewidth=2, label='Threshold (2.0)')
ax2.axvspan(2.0, 2.5, alpha=0.2, color='green', label='Pure sample')
ax2.set_xlabel('A260/A230 Ratio', fontsize=12, fontweight='bold')
ax2.set_title('Purity Assessment: Salt/Organic Contamination', fontsize=14, fontweight='bold')
ax2.legend(loc='lower right')
ax2.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Purity indicators visualized!")
print("   Green bars = Pure samples")
print("   Red/Orange bars = Contaminated samples")

---
## Practice 5: Fluorescence Spectra Analysis

### 🎯 Learning Objectives
- Simulate excitation and emission spectra
- Calculate Stokes shift
- Visualize fluorophore properties

### 📖 Key Concepts
- **Stokes shift:** λ_emission > λ_excitation (typically 20-100 nm)
- **Quantum yield (Φ):** photons emitted / photons absorbed
- **Brightness:** ε × Φ

In [ ]:
# 5.1 Simulate fluorophore spectra
def gaussian_spectrum(wavelengths, peak, width, amplitude=1.0):
    """Generate Gaussian-shaped spectrum"""
    return amplitude * np.exp(-((wavelengths - peak)**2) / (2 * width**2))

# Define common fluorophores
fluorophores = {
    'GFP': {
        'ex_peak': 488, 'ex_width': 25,
        'em_peak': 509, 'em_width': 30,
        'QY': 0.79, 'epsilon': 55000,
        'color': '#27ae60'
    },
    'mCherry': {
        'ex_peak': 587, 'ex_width': 30,
        'em_peak': 610, 'em_width': 35,
        'QY': 0.22, 'epsilon': 72000,
        'color': '#E74C3C'
    },
    'CFP': {
        'ex_peak': 433, 'ex_width': 28,
        'em_peak': 475, 'em_width': 32,
        'QY': 0.40, 'epsilon': 32500,
        'color': '#3498db'
    },
    'YFP': {
        'ex_peak': 514, 'ex_width': 27,
        'em_peak': 527, 'em_width': 30,
        'QY': 0.61, 'epsilon': 83400,
        'color': '#f39c12'
    }
}

wavelengths = np.linspace(350, 700, 1000)

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

print("🌈 Fluorescent Protein Spectral Properties")
print("=" * 80)
print(f"{'Fluorophore':<12} {'Ex peak':<10} {'Em peak':<10} {'Stokes':<10} {'QY':<10} {'Brightness'}")
print("=" * 80)

for idx, (name, props) in enumerate(fluorophores.items()):
    ax = axes[idx]
    
    # Generate spectra
    excitation = gaussian_spectrum(wavelengths, props['ex_peak'], props['ex_width'])
    emission = gaussian_spectrum(wavelengths, props['em_peak'], props['em_width'])
    
    # Calculate Stokes shift
    stokes_shift = props['em_peak'] - props['ex_peak']
    
    # Calculate brightness
    brightness = props['epsilon'] * props['QY'] / 1000  # Normalized
    
    # Plot
    ax.fill_between(wavelengths, 0, excitation, alpha=0.3, color=props['color'], label='Excitation')
    ax.fill_between(wavelengths, 0, emission, alpha=0.5, color=props['color'], label='Emission')
    ax.plot(wavelengths, excitation, color=props['color'], linewidth=2, linestyle='--')
    ax.plot(wavelengths, emission, color=props['color'], linewidth=2.5)
    
    # Mark peaks
    ax.axvline(props['ex_peak'], color='blue', linestyle=':', alpha=0.7, linewidth=1.5)
    ax.axvline(props['em_peak'], color='red', linestyle=':', alpha=0.7, linewidth=1.5)
    
    # Stokes shift arrow
    ax.annotate('', xy=(props['em_peak'], 0.5), xytext=(props['ex_peak'], 0.5),
                arrowprops=dict(arrowstyle='<->', color='black', lw=2))
    ax.text((props['ex_peak'] + props['em_peak'])/2, 0.55, f'{stokes_shift} nm',
            ha='center', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Wavelength (nm)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Normalized Intensity', fontsize=11, fontweight='bold')
    ax.set_title(f'{name} (Φ = {props["QY"]:.2f}, Brightness = {brightness:.1f})', 
                 fontsize=13, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(350, 700)
    
    # Print table
    print(f"{name:<12} {props['ex_peak']:<10} {props['em_peak']:<10} "
          f"{stokes_shift:<10} {props['QY']:<10.2f} {brightness:.1f}")

plt.tight_layout()
plt.show()

print("\n✨ Stokes shift enables detection of emission without excitation interference!")

---
## Practice 6: FRET Efficiency Calculation

### 🎯 Learning Objectives
- Calculate FRET efficiency from distance
- Understand Förster radius (R₀)
- Apply 1/r⁶ distance dependence

### 📖 Key Formula
**E = R₀⁶ / (R₀⁶ + r⁶)**
- E: FRET efficiency
- R₀: Förster radius (typically 2-10 nm)
- r: Donor-acceptor distance

In [ ]:
# 6.1 FRET efficiency calculations
def calculate_fret_efficiency(distance, R0):
    """Calculate FRET efficiency using Förster equation"""
    return R0**6 / (R0**6 + distance**6)

def distance_from_fret_efficiency(efficiency, R0):
    """Calculate distance from measured FRET efficiency"""
    return R0 * ((1/efficiency - 1)**(1/6))

# Common FRET pairs
fret_pairs = {
    'CFP-YFP': {'R0': 4.9, 'color': '#3498db'},
    'GFP-mCherry': {'R0': 5.6, 'color': '#27ae60'},
    'Alexa488-Alexa594': {'R0': 6.3, 'color': '#f39c12'},
    'Cy3-Cy5': {'R0': 5.4, 'color': '#9b59b6'}
}

# Distance range
distances = np.linspace(1, 15, 200)

# Plot FRET efficiency vs distance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: FRET efficiency curves
for pair_name, props in fret_pairs.items():
    R0 = props['R0']
    efficiencies = [calculate_fret_efficiency(d, R0) for d in distances]
    
    ax1.plot(distances, efficiencies, linewidth=3, color=props['color'], 
             label=f"{pair_name} (R₀={R0} nm)")
    
    # Mark 50% efficiency point
    ax1.plot(R0, 0.5, 'o', markersize=10, color=props['color'], 
             markeredgecolor='black', markeredgewidth=2)

ax1.axhline(0.5, color='gray', linestyle='--', linewidth=2, alpha=0.5, label='50% efficiency')
ax1.set_xlabel('Distance (nm)', fontsize=13, fontweight='bold')
ax1.set_ylabel('FRET Efficiency', fontsize=13, fontweight='bold')
ax1.set_title('FRET Efficiency: E = R₀⁶/(R₀⁶ + r⁶)', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(1, 15)
ax1.set_ylim(0, 1)

# Annotate key point
ax1.annotate('When r = R₀,\nE = 50%', xy=(5.5, 0.5), xytext=(8, 0.7),
            arrowprops=dict(arrowstyle='->', color='black', lw=2),
            fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Plot 2: Sensitivity analysis (1/r⁶ dependence)
R0_example = 5.5
distances_fine = np.linspace(2, 12, 100)
efficiencies_fine = [calculate_fret_efficiency(d, R0_example) for d in distances_fine]

# Calculate sensitivity (dE/dr)
sensitivity = np.gradient(efficiencies_fine, distances_fine)

ax2_twin = ax2.twinx()
line1 = ax2.plot(distances_fine, efficiencies_fine, 'b-', linewidth=3, label='FRET Efficiency')
line2 = ax2_twin.plot(distances_fine, np.abs(sensitivity), 'r--', linewidth=3, label='Sensitivity |dE/dr|')

ax2.axvline(R0_example, color='green', linestyle=':', linewidth=2, alpha=0.7)
ax2.set_xlabel('Distance (nm)', fontsize=13, fontweight='bold')
ax2.set_ylabel('FRET Efficiency', fontsize=13, fontweight='bold', color='blue')
ax2_twin.set_ylabel('Sensitivity |dE/dr|', fontsize=13, fontweight='bold', color='red')
ax2.set_title('Distance Sensitivity of FRET', fontsize=14, fontweight='bold')
ax2.tick_params(axis='y', labelcolor='blue')
ax2_twin.tick_params(axis='y', labelcolor='red')
ax2.grid(True, alpha=0.3)

# Combined legend
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax2.legend(lines, labels, loc='upper right')

plt.tight_layout()
plt.show()

print("\n💡 FRET is most sensitive to distance changes around R₀!")
print("   This 1/r⁶ dependence makes FRET a 'molecular ruler' for 2-10 nm range.")

In [ ]:
# 6.2 Practical FRET measurement example
print("🔬 FRET Biosensor Experiment Simulation")
print("=" * 70)

# Experimental scenario: Calcium sensor
R0_sensor = 5.0  # nm

# Different calcium concentrations cause conformational changes
ca_concentrations = [0, 0.1, 0.5, 1.0, 5.0, 10.0]  # μM
# Distance changes with calcium binding
distances_ca = [8.5, 7.8, 6.5, 5.5, 4.8, 4.5]  # nm

# Calculate FRET efficiencies
fret_efficiencies = [calculate_fret_efficiency(d, R0_sensor) for d in distances_ca]

# Simulate donor and acceptor intensities
donor_intensities = [100 * (1 - E) for E in fret_efficiencies]
acceptor_intensities = [100 * E for E in fret_efficiencies]

print(f"\n{'[Ca²⁺] (μM)':<15} {'Distance (nm)':<15} {'FRET E':<12} {'Donor I':<12} {'Acceptor I'}")
print("=" * 70)
for ca, dist, eff, donor, acceptor in zip(ca_concentrations, distances_ca, 
                                            fret_efficiencies, donor_intensities, 
                                            acceptor_intensities):
    print(f"{ca:<15.1f} {dist:<15.2f} {eff:<12.3f} {donor:<12.1f} {acceptor:.1f}")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: FRET efficiency vs calcium
ax1.plot(ca_concentrations, fret_efficiencies, 'o-', linewidth=3, markersize=10,
         color='#9b59b6', markeredgecolor='black', markeredgewidth=2)
ax1.set_xlabel('[Ca²⁺] (μM)', fontsize=13, fontweight='bold')
ax1.set_ylabel('FRET Efficiency', fontsize=13, fontweight='bold')
ax1.set_title('Calcium Biosensor Response', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xscale('log')
ax1.set_xlim(0.05, 15)

# Plot 2: Donor and acceptor intensities
x_pos = np.arange(len(ca_concentrations))
width = 0.35

bars1 = ax2.bar(x_pos - width/2, donor_intensities, width, label='Donor (CFP)',
                color='#3498db', alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax2.bar(x_pos + width/2, acceptor_intensities, width, label='Acceptor (YFP)',
                color='#f39c12', alpha=0.8, edgecolor='black', linewidth=1.5)

ax2.set_xlabel('[Ca²⁺] (μM)', fontsize=13, fontweight='bold')
ax2.set_ylabel('Fluorescence Intensity (a.u.)', fontsize=13, fontweight='bold')
ax2.set_title('Donor Quenching and Acceptor Sensitization', fontsize=14, fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f'{c:.1f}' for c in ca_concentrations])
ax2.legend()
ax2.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ As calcium increases:")
print("   → Distance decreases (conformational change)")
print("   → FRET efficiency increases")
print("   → Donor intensity decreases (quenching)")
print("   → Acceptor intensity increases (sensitization)")

---
## Practice 7: Spectral Data Processing

### 🎯 Learning Objectives
- Apply baseline correction
- Perform peak detection
- Smooth noisy spectral data
- Calculate signal-to-noise ratio (SNR)

### 📖 Methods
- **Savitzky-Golay filter:** Polynomial smoothing
- **Baseline correction:** Polynomial fitting
- **Peak detection:** scipy.signal.find_peaks

In [ ]:
# 7.1 Generate realistic noisy spectrum
np.random.seed(42)

# Generate wavelength range
wavelengths_spec = np.linspace(400, 700, 500)

# True spectrum: multiple Gaussian peaks
true_spectrum = (gaussian_spectrum(wavelengths_spec, 450, 20, 0.8) +
                 gaussian_spectrum(wavelengths_spec, 520, 25, 1.0) +
                 gaussian_spectrum(wavelengths_spec, 600, 30, 0.6))

# Add baseline drift (polynomial)
baseline = 0.1 + 0.0002 * wavelengths_spec + 0.0000005 * wavelengths_spec**2

# Add noise
noise_level = 0.03
noise = np.random.normal(0, noise_level, len(wavelengths_spec))

# Observed spectrum
observed_spectrum = true_spectrum + baseline + noise

# Calculate SNR
signal_power = np.mean(true_spectrum**2)
noise_power = np.mean(noise**2)
SNR_dB = 10 * np.log10(signal_power / noise_power)

print(f"📊 Simulated Spectrum Statistics:")
print(f"   Signal-to-Noise Ratio: {SNR_dB:.2f} dB")
print(f"   Noise level: {noise_level:.3f}")

In [ ]:
# 7.2 Apply signal processing techniques
from scipy.signal import savgol_filter, find_peaks

# Step 1: Baseline correction (polynomial fit)
baseline_coeffs = np.polyfit(wavelengths_spec, observed_spectrum, 2)
baseline_fit = np.polyval(baseline_coeffs, wavelengths_spec)
spectrum_baseline_corrected = observed_spectrum - baseline_fit

# Step 2: Smoothing (Savitzky-Golay filter)
spectrum_smoothed = savgol_filter(spectrum_baseline_corrected, window_length=15, polyorder=3)

# Step 3: Peak detection
peaks, properties = find_peaks(spectrum_smoothed, height=0.3, distance=30, prominence=0.2)
peak_wavelengths = wavelengths_spec[peaks]
peak_heights = spectrum_smoothed[peaks]

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Raw data with noise
axes[0, 0].plot(wavelengths_spec, observed_spectrum, 'gray', alpha=0.5, linewidth=1, label='Raw data')
axes[0, 0].plot(wavelengths_spec, true_spectrum + baseline, 'b-', linewidth=2, label='True signal')
axes[0, 0].set_xlabel('Wavelength (nm)', fontweight='bold')
axes[0, 0].set_ylabel('Intensity', fontweight='bold')
axes[0, 0].set_title('Step 0: Raw Noisy Spectrum', fontsize=13, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Baseline correction
axes[0, 1].plot(wavelengths_spec, observed_spectrum, 'gray', alpha=0.5, linewidth=1, label='Raw data')
axes[0, 1].plot(wavelengths_spec, baseline_fit, 'r--', linewidth=2, label='Baseline fit')
axes[0, 1].plot(wavelengths_spec, spectrum_baseline_corrected, 'g-', linewidth=1.5, 
                label='Baseline corrected')
axes[0, 1].set_xlabel('Wavelength (nm)', fontweight='bold')
axes[0, 1].set_ylabel('Intensity', fontweight='bold')
axes[0, 1].set_title('Step 1: Baseline Correction', fontsize=13, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Smoothing
axes[1, 0].plot(wavelengths_spec, spectrum_baseline_corrected, 'gray', alpha=0.5, 
                linewidth=1, label='Noisy')
axes[1, 0].plot(wavelengths_spec, spectrum_smoothed, 'b-', linewidth=2.5, label='Smoothed')
axes[1, 0].plot(wavelengths_spec, true_spectrum, 'r--', linewidth=2, alpha=0.7, label='True signal')
axes[1, 0].set_xlabel('Wavelength (nm)', fontweight='bold')
axes[1, 0].set_ylabel('Intensity', fontweight='bold')
axes[1, 0].set_title('Step 2: Savitzky-Golay Smoothing', fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Peak detection
axes[1, 1].plot(wavelengths_spec, spectrum_smoothed, 'b-', linewidth=2.5, label='Processed')
axes[1, 1].plot(peak_wavelengths, peak_heights, 'r*', markersize=20, 
                markeredgecolor='black', markeredgewidth=1.5, label=f'Detected peaks ({len(peaks)})')

# Annotate peaks
for wl, h in zip(peak_wavelengths, peak_heights):
    axes[1, 1].annotate(f'{wl:.0f} nm', xy=(wl, h), xytext=(wl, h+0.15),
                       ha='center', fontsize=10, fontweight='bold',
                       arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

axes[1, 1].set_xlabel('Wavelength (nm)', fontweight='bold')
axes[1, 1].set_ylabel('Intensity', fontweight='bold')
axes[1, 1].set_title('Step 3: Peak Detection', fontsize=13, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🎯 Peak Detection Results:")
print(f"   Number of peaks detected: {len(peaks)}")
print(f"\n   Peak positions and intensities:")
for i, (wl, h) in enumerate(zip(peak_wavelengths, peak_heights), 1):
    print(f"   Peak {i}: λ = {wl:.1f} nm, Intensity = {h:.3f}")

print("\n✅ Signal processing pipeline complete!")

---
## Practice 8: Flow Cytometry Data Simulation

### 🎯 Learning Objectives
- Simulate flow cytometry scatter and fluorescence data
- Perform gating analysis
- Visualize multi-parameter data

### 📖 Key Parameters
- **FSC:** Forward scatter (cell size)
- **SSC:** Side scatter (granularity)
- **FL1-FLn:** Fluorescence channels

In [ ]:
# 8.1 Simulate flow cytometry data
np.random.seed(42)

def simulate_cell_population(n_cells, mean_fsc, mean_ssc, mean_fl, std_factor=0.2):
    """Simulate a cell population with given characteristics"""
    fsc = np.random.lognormal(np.log(mean_fsc), std_factor, n_cells)
    ssc = np.random.lognormal(np.log(mean_ssc), std_factor, n_cells)
    fl1 = np.random.lognormal(np.log(mean_fl), std_factor, n_cells)
    return fsc, ssc, fl1

# Simulate different cell populations
# Population 1: Lymphocytes (small, low granularity, GFP-)
n_lymph = 5000
fsc_lymph, ssc_lymph, fl1_lymph = simulate_cell_population(n_lymph, 300, 200, 50)

# Population 2: Monocytes (medium, medium granularity, GFP-)
n_mono = 2000
fsc_mono, ssc_mono, fl1_mono = simulate_cell_population(n_mono, 500, 400, 60)

# Population 3: Granulocytes (large, high granularity, GFP-)
n_gran = 1500
fsc_gran, ssc_gran, fl1_gran = simulate_cell_population(n_gran, 700, 800, 55)

# Population 4: GFP+ cells (transfected lymphocytes)
n_gfp = 800
fsc_gfp, ssc_gfp, fl1_gfp = simulate_cell_population(n_gfp, 320, 220, 800, std_factor=0.3)

# Combine all populations
fsc_all = np.concatenate([fsc_lymph, fsc_mono, fsc_gran, fsc_gfp])
ssc_all = np.concatenate([ssc_lymph, ssc_mono, ssc_gran, ssc_gfp])
fl1_all = np.concatenate([fl1_lymph, fl1_mono, fl1_gran, fl1_gfp])
labels = (['Lymphocyte']*n_lymph + ['Monocyte']*n_mono + 
          ['Granulocyte']*n_gran + ['GFP+']*n_gfp)

# Create DataFrame
flow_data = pd.DataFrame({
    'FSC': fsc_all,
    'SSC': ssc_all,
    'FL1_GFP': fl1_all,
    'Population': labels
})

print("🔬 Flow Cytometry Data Simulation")
print("=" * 60)
print(f"Total events: {len(flow_data):,}")
print("\nPopulation distribution:")
print(flow_data['Population'].value_counts())
print("\nData preview:")
print(flow_data.head(10))

In [ ]:
# 8.2 Visualize flow cytometry data with gating
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# Define colors for populations
pop_colors = {
    'Lymphocyte': '#3498db',
    'Monocyte': '#27ae60',
    'Granulocyte': '#e74c3c',
    'GFP+': '#f39c12'
}

# Plot 1: FSC vs SSC (all events)
ax1 = fig.add_subplot(gs[0, 0])
for pop in flow_data['Population'].unique():
    mask = flow_data['Population'] == pop
    ax1.scatter(flow_data.loc[mask, 'FSC'], flow_data.loc[mask, 'SSC'], 
                s=2, alpha=0.4, c=pop_colors[pop], label=pop)
ax1.set_xlabel('FSC (Forward Scatter)', fontsize=11, fontweight='bold')
ax1.set_ylabel('SSC (Side Scatter)', fontsize=11, fontweight='bold')
ax1.set_title('FSC vs SSC - All Events', fontsize=13, fontweight='bold')
ax1.legend(markerscale=3)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Plot 2: Density plot FSC vs SSC
ax2 = fig.add_subplot(gs[0, 1])
h = ax2.hexbin(flow_data['FSC'], flow_data['SSC'], gridsize=50, cmap='YlOrRd', 
               xscale='log', yscale='log', mincnt=1)
ax2.set_xlabel('FSC (Forward Scatter)', fontsize=11, fontweight='bold')
ax2.set_ylabel('SSC (Side Scatter)', fontsize=11, fontweight='bold')
ax2.set_title('Density Plot - Cell Distribution', fontsize=13, fontweight='bold')
plt.colorbar(h, ax=ax2, label='Event Count')
ax2.grid(True, alpha=0.3)

# Plot 3: GFP fluorescence histogram
ax3 = fig.add_subplot(gs[0, 2])
for pop in ['Lymphocyte', 'GFP+']:
    mask = flow_data['Population'] == pop
    data_subset = flow_data.loc[mask, 'FL1_GFP']
    ax3.hist(data_subset, bins=50, alpha=0.6, label=pop, color=pop_colors[pop], edgecolor='black')

# Add gate threshold
gate_threshold = 200
ax3.axvline(gate_threshold, color='red', linestyle='--', linewidth=3, label=f'Gate ({gate_threshold})')
ax3.set_xlabel('FL1 - GFP Intensity', fontsize=11, fontweight='bold')
ax3.set_ylabel('Event Count', fontsize=11, fontweight='bold')
ax3.set_title('GFP Fluorescence Distribution', fontsize=13, fontweight='bold')
ax3.legend()
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3)

# Plot 4: FSC vs GFP (gating)
ax4 = fig.add_subplot(gs[1, 0])
gfp_negative = flow_data['FL1_GFP'] < gate_threshold
gfp_positive = flow_data['FL1_GFP'] >= gate_threshold

ax4.scatter(flow_data.loc[gfp_negative, 'FSC'], flow_data.loc[gfp_negative, 'FL1_GFP'],
            s=2, alpha=0.3, c='gray', label='GFP-')
ax4.scatter(flow_data.loc[gfp_positive, 'FSC'], flow_data.loc[gfp_positive, 'FL1_GFP'],
            s=5, alpha=0.6, c='#f39c12', edgecolors='black', linewidth=0.2, label='GFP+')
ax4.axhline(gate_threshold, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax4.set_xlabel('FSC (Forward Scatter)', fontsize=11, fontweight='bold')
ax4.set_ylabel('FL1 - GFP Intensity', fontsize=11, fontweight='bold')
ax4.set_title('FSC vs GFP - Gating Analysis', fontsize=13, fontweight='bold')
ax4.legend(markerscale=3)
ax4.set_yscale('log')
ax4.set_xscale('log')
ax4.grid(True, alpha=0.3)

# Plot 5: Population statistics
ax5 = fig.add_subplot(gs[1, 1])
pop_counts = flow_data['Population'].value_counts()
colors_bar = [pop_colors[pop] for pop in pop_counts.index]
bars = ax5.bar(range(len(pop_counts)), pop_counts.values, color=colors_bar, 
               alpha=0.8, edgecolor='black', linewidth=2)
ax5.set_xticks(range(len(pop_counts)))
ax5.set_xticklabels(pop_counts.index, rotation=45, ha='right')
ax5.set_ylabel('Event Count', fontsize=11, fontweight='bold')
ax5.set_title('Population Counts', fontsize=13, fontweight='bold')
ax5.grid(True, axis='y', alpha=0.3)

# Add percentage labels
total = len(flow_data)
for i, (bar, count) in enumerate(zip(bars, pop_counts.values)):
    percentage = (count / total) * 100
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
             f'{percentage:.1f}%', ha='center', fontsize=10, fontweight='bold')

# Plot 6: Gating statistics
ax6 = fig.add_subplot(gs[1, 2])
n_positive = gfp_positive.sum()
n_negative = gfp_negative.sum()
percent_positive = (n_positive / total) * 100

gate_stats = pd.DataFrame({
    'Gate': ['GFP-', 'GFP+'],
    'Count': [n_negative, n_positive],
    'Percentage': [(n_negative/total)*100, percent_positive]
})

colors_gate = ['gray', '#f39c12']
bars2 = ax6.bar(gate_stats['Gate'], gate_stats['Count'], color=colors_gate,
                alpha=0.8, edgecolor='black', linewidth=2)
ax6.set_ylabel('Event Count', fontsize=11, fontweight='bold')
ax6.set_title('Gating Analysis Results', fontsize=13, fontweight='bold')
ax6.grid(True, axis='y', alpha=0.3)

for bar, pct in zip(bars2, gate_stats['Percentage']):
    ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100,
             f'{pct:.2f}%', ha='center', fontsize=11, fontweight='bold')

plt.suptitle('Flow Cytometry Analysis Dashboard', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("\n📊 Gating Analysis Summary:")
print("=" * 60)
print(f"Gate threshold: FL1 > {gate_threshold}")
print(f"\nGFP-negative cells: {n_negative:,} ({(n_negative/total)*100:.2f}%)")
print(f"GFP-positive cells: {n_positive:,} ({percent_positive:.2f}%)")
print(f"\n✅ Transfection efficiency: {percent_positive:.2f}%")

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **Energy-Wavelength Calculations**: Applied Planck-Einstein relation to biological systems
2. **Beer-Lambert Law**: Quantitative absorbance spectroscopy for concentration determination
3. **Protein Quantification**: A280, Bradford, and BCA assay methods
4. **Nucleic Acid Analysis**: DNA/RNA quantification and purity assessment (A260/A280)
5. **Fluorescence Spectroscopy**: Stokes shift, quantum yield, and spectral properties
6. **FRET**: Distance-dependent energy transfer and biosensor applications
7. **Signal Processing**: Baseline correction, smoothing, and peak detection
8. **Flow Cytometry**: Multi-parameter cell analysis and gating strategies

### Key Insights:
- Spectroscopy provides quantitative, non-destructive measurements
- Energy relationships govern photon-matter interactions
- FRET enables nanometer-scale distance measurements in living cells
- Flow cytometry allows rapid single-cell multiparameter analysis

### Next Steps:
- Advanced microscopy techniques
- Super-resolution imaging (PALM, STORM)
- Time-resolved spectroscopy
- Machine learning for spectral analysis